In [1]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 103.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [10]:
# get dataset from roboflow
from roboflow import Roboflow

rf = Roboflow(api_key="HTjMDkIgyNz0OGcJh7a9")

# Make sure project slug is correct
project = rf.workspace("fashion-by4cb").project("eye-color-bc6ji")

# Version 1 or 4 works (use 4 if you want the most recent)
dataset = project.version(4).download("folder")

print("Dataset downloaded to:", dataset.location)

loading Roboflow workspace...
loading Roboflow project...
Dataset downloaded to: /content/Eye-Color-1


In [11]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

# Data generators with rescaling
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen   = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

# Point to the actual dataset folders
train_dir = "Eye-Color-4/train"
val_dir   = "Eye-Color-4/valid"
test_dir  = "Eye-Color-4/test"

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

# Build a small CNN
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(*IMG_SIZE, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(train_gen.num_classes, activation="softmax"),
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
history = model.fit(train_gen, validation_data=val_gen, epochs=20)

Found 332 images belonging to 6 classes.
Found 11 images belonging to 6 classes.
Found 10 images belonging to 5 classes.


Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.


Epoch 1/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 980ms/step - accuracy: 0.2203 - loss: 2.3284 - val_accuracy: 0.2727 - val_loss: 1.7897
Epoch 2/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 866ms/step - accuracy: 0.4252 - loss: 1.4166 - val_accuracy: 0.3636 - val_loss: 1.9964
Epoch 3/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 807ms/step - accuracy: 0.5469 - loss: 1.1469 - val_accuracy: 0.1818 - val_loss: 2.1250
Epoch 4/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 975ms/step - accuracy: 0.6311 - loss: 0.8957 - val_accuracy: 0.2727 - val_loss: 1.7287
Epoch 5/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 928ms/step - accuracy: 0.7136 - loss: 0.7345 - val_accuracy: 0.3636 - val_loss: 2.1378
Epoch 6/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 761ms/step - accuracy: 0.8128 - loss: 0.5276 - val_accuracy: 0.3636 - val_loss: 2.1412
Epoch 7/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 958ms/step - accuracy: 0.7782 - loss: 0.4662 - val_accuracy: 0.2727 - val_loss: 2.1224
Epoch 8/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 944ms/step - accuracy: 0.9127 - loss: 0.3093 - val_accur

In [12]:
# Map index → label for predictions later
class_indices = train_gen.class_indices
idx_to_class = {v:k for k,v in class_indices.items()}
print("Class mapping:", idx_to_class)

Class mapping: {0: 'amber', 1: 'blue', 2: 'brown', 3: 'green', 4: 'grey', 5: 'hazel'}


In [13]:
# Save for tfjs or later use
model.save("eye_color_v0.h5")